In [1]:
import ants
import os
import glob
import nipype.interfaces.io as nio
import time

GROUPS=['PD','ET','NC']
SUB_LIST=[]; AGE_LIST=[];
current_group = GROUPS[0]
current_sub_list_file = '/codes/devel/PD_Marker/'+current_group+'_info_ICA.list'
TEMPLATE_IND = '%s_desc-masked_T1w.nii.gz'
TEMPLATE_TMP = '%s_space-MNI152NLin2009cAsym_desc-masked_T1w.nii.gz'

T1_DIR='/output/masked_T1/'+current_group+'/'
TMP_OUT_DIR='/output/group_template/'+current_group+'_group_template/'
if not os.path.exists(TMP_OUT_DIR):
    os.makedirs(TMP_OUT_DIR)

with open(current_sub_list_file, 'r') as f_sub:
    sub_list_raw= f_sub.readlines()
sub_list = [x[0:-1].split('\t')[0] for x in sub_list_raw] # remove 

# dataGrabber
ds_masked_t1 = nio.DataGrabber(infields=['sub_id'])
ds_masked_t1.inputs.base_directory = T1_DIR # database
ds_masked_t1.inputs.template = TEMPLATE_TMP # from cwd
ds_masked_t1.inputs.sort_filelist = True
ds_masked_t1.inputs.sub_id = sub_list
res_masked_t1 = ds_masked_t1.run()
group_files=res_masked_t1.outputs.outfiles
group_images = [ants.image_read(x, dimension = 3) for x in group_files]
print(group_files)
t0=time.time()
btp = ants.build_template(initialTemplate = None,
  image_list = group_images,
  iterations = 4,
  gradient_step = 0.2,
  verbose = True,
  syn_metric = 'CC',
  type_of_transform = 'SyN' )
ants.plot(btp)
print('PD template construction takes: ', time.time()-t0)


Bad key "text.kerning_factor" on line 4 in
/usr/local/miniconda/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


201023-21:55:42,428 nipype.utils WARNING:
	 A newer version (1.5.1) of nipy/nipype is available. You are using 1.5.0


KeyboardInterrupt: 

In [2]:
# register ET template to MNI2009C
# basic functions
from nipype.interfaces.ants import ANTS
import os,time
OUT_DIR='/output/tmp_template/'
if not os.path.exists(OUT_DIR):
    os.makedirs(OUT_DIR)
atlas_09_masked='/templateflow/tpl-MNI152NLin2009cAsym/tpl-MNI152NLin2009cAsym_res-01_desc-brain_T1w.nii.gz'
ET_TEMPLATE='/output/masked_T1/ET/individual/T_template0.nii.gz'
import time
import copy, pprint
from nipype.interfaces.ants import Registration
t0=time.time()
reg = Registration()
reg.inputs.fixed_image = atlas_09_masked
reg.inputs.moving_image = ET_TEMPLATE
reg.inputs.output_transform_prefix = OUT_DIR+'output_'
#reg.inputs.initial_moving_transform = 'trans.mat'
reg.inputs.transforms = ['Affine', 'SyN']
reg.inputs.transform_parameters = [(2.0,), (0.25, 3.0, 0.0)]
reg.inputs.number_of_iterations = [[1500, 200], [100, 50, 30]]
reg.inputs.dimension = 3
reg.inputs.write_composite_transform = True
reg.inputs.collapse_output_transforms = False
reg.inputs.initialize_transforms_per_stage = False
reg.inputs.metric = ['CC']*2
reg.inputs.metric_weight = [1]*2 # Default (value ignored currently by ANTs)
reg.inputs.radius_or_number_of_bins = [32]*2
reg.inputs.sampling_strategy = ['Random', None]
reg.inputs.sampling_percentage = [0.05, None]
reg.inputs.convergence_threshold = [1.e-8, 1.e-9]
reg.inputs.convergence_window_size = [20]*2
reg.inputs.smoothing_sigmas = [[1,0], [2,1,0]]
reg.inputs.sigma_units = ['vox'] * 2
reg.inputs.shrink_factors = [[2,1], [3,2,1]]
reg.inputs.use_estimate_learning_rate_once = [True, True]
reg.inputs.use_histogram_matching = [True, True] # This is the default
reg.inputs.output_warped_image = OUT_DIR+'ET_mni2009c_template.nii.gz'
reg.cmdline
reg.run()  
print('Reg to mni template takes: ',time.time()-t0)

201016-02:23:34,678 nipype.interface INFO:
	 stdout 2020-10-16T02:23:34.677336:All_Command_lines_OK
201016-02:23:34,683 nipype.interface INFO:
	 stdout 2020-10-16T02:23:34.683544:Using double precision for computations.
201016-02:23:34,690 nipype.interface INFO:
	 stdout 2020-10-16T02:23:34.689991:  number of levels = 2
201016-02:23:34,693 nipype.interface INFO:
	 stdout 2020-10-16T02:23:34.689991:  number of levels = 3
201016-02:23:34,694 nipype.interface INFO:
	 stdout 2020-10-16T02:23:34.689991:  fixed image: /templateflow/tpl-MNI152NLin2009cAsym/tpl-MNI152NLin2009cAsym_res-01_desc-brain_T1w.nii.gz
201016-02:23:34,697 nipype.interface INFO:
	 stdout 2020-10-16T02:23:34.689991:  moving image: /output/masked_T1/ET/individual/T_template0.nii.gz
201016-02:23:35,559 nipype.interface INFO:
	 stdout 2020-10-16T02:23:35.558933:  fixed image: /templateflow/tpl-MNI152NLin2009cAsym/tpl-MNI152NLin2009cAsym_res-01_desc-brain_T1w.nii.gz
201016-02:23:35,560 nipype.interface INFO:
	 stdout 2020-10-

KeyboardInterrupt: 

In [ ]:
# Masking PD/ET/NC group images
import os
import time
from nipype.interfaces import fsl

def dataGraber_sub(SUB_ID, DATA_DIR, TMPT_STR):
    import nipype.interfaces.io as nio
    import time
    t0=time.time()
    print('Grabbing files for: ', SUB_ID)
    OUT_FILE=[]
    out_len=len(TMPT_STR)
    if out_len == 0:
        print(SUB_ID+' has no files named: ', TMPT_STR)
        return OUT_FILE
    else:
        for i in range(out_len):
            TMP='%s/anat/%s_'+TMPT_STR[i]
            ds = nio.DataGrabber(infields=['subject_id', 'subject_id'])
            ds.inputs.base_directory = DATA_DIR # database
            ds.inputs.template = TMP 
            ds.inputs.subject_id = [SUB_ID]
            ds.inputs.sort_filelist = True
            res = ds.run()
            res_list = res.outputs.outfiles
            OUT_FILE.append(res_list)
        #print(SUB_ID+' files: ', OUT_FILE)
        print('dataGraber takes: ', time.time()-t0 )
        return OUT_FILE
## masking all group subjects
GROUPS=['PD','ET','NC']
SUB_LIST=[]; AGE_LIST=[];
OUT_DIR='/output/masked_T1/'

for group_name in GROUPS:
    current_group=group_name
    current_sub_list_file = '/codes/devel/PD_Marker/'+current_group+'_info_ICA.list'
    # create dir for output
    current_OUT_DIR=OUT_DIR+current_group+'/'
    current_ind_OUT_DIR = current_OUT_DIR+'individual/'
    current_tmp_OUT_DIR = current_OUT_DIR+'mni2009cAsym/'
    if not os.path.exists(current_ind_OUT_DIR):
        os.makedirs(current_ind_OUT_DIR)
    if not os.path.exists(current_tmp_OUT_DIR):
        os.makedirs(current_tmp_OUT_DIR)
    #read sub list
    with open(current_sub_list_file, 'r') as f_sub:
        sub_list_raw= f_sub.readlines()
    sub_list = [x[0:-1].split('\t')[0] for x in sub_list_raw] # remove 
    age_list = [x[0:-1].split('\t')[1] for x in sub_list_raw]
    SUB_LIST.append(sub_list);  AGE_LIST.append(age_list);
    N_sub=len(sub_list)
    print(N_sub, sub_list, age_list)
    DATA_DIR='/data/'+current_group
    TMPT_list=['desc-preproc_T1w.nii.gz', \
               'desc-brain_mask.nii.gz', \
               'space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz', \
               'space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz'];
    print('Masking ',current_group,' group starts:')
    t0=time.time()
    for i_sub in range(0, N_sub,1):
        sub_str=sub_list[i_sub]
        print('Masking ',sub_str,' ( ',i_sub , ' )...')
        res1 = dataGraber_sub(sub_str, DATA_DIR, TMPT_list)
        ind_t1 = res1[0]; ind_bm = res1[1];
        tmp_t1 = res1[2]; tmp_bm = res1[3];
        ind_masked_t1 = current_ind_OUT_DIR+sub_str+'_'+'desc-masked_T1w.nii.gz'
        tmp_masked_t1 = current_tmp_OUT_DIR+sub_str+'_'+'space-MNI152NLin2009cAsym_desc-masked_T1w.nii.gz'
        mask_ind = fsl.ApplyMask(
            in_file=ind_t1,
            out_file=ind_masked_t1,
            mask_file=ind_bm)
        mask_ind.run()
        mask_tmp = fsl.ApplyMask(
            in_file=tmp_t1,
            out_file=tmp_masked_t1,
            mask_file=tmp_bm)
        mask_tmp.run()
    print('Masking ',current_group,' finished in ',time.time()-t0, '\n')